In [1]:
import pandas
import psycopg2
from getpass import getpass
from pandas import read_sql

In [2]:
#Candidate password 
p = getpass('password')

password········


In [3]:
#FindHotel Amazon Redshift access details
config = { 'dbname': 'datafeeds', 
           'user':'candidate',
           'pwd': p,
           'host':'datafeeds-1.c1ni7c7e0d3e.eu-west-1.redshift.amazonaws.com',
           'port':'5439'
         }

In [4]:
#Creating connection with Amazon Redshift 
def create_connection(*args,**kwargs):
    config = kwargs['config']
    con=psycopg2.connect(dbname=config['dbname'], host=config['host'], port=config['port'], user=config['user'], password=config['pwd'])
    return con
    
connection = create_connection(config=config)
print "Connection with FH AWS Redshift DW established"

Connection with FH AWS Redshift DW established


In [5]:
#Extracting place table and creating local csv
place = read_sql("select * from toy1.place", con=connection)
place.to_csv('place.csv', index = False)
place.head(2)

,place_id,place_name,place_file_name,place_type_id,country_code,latitude,longitude,rank,navigation_path,is_searchable,has_image,sitemap,wk_place_id,hotel_count_wb,hotel_count_mi,hotel_count_km,has_generic_image,site_map_ancestor
0,1,Egyhazasradoc,Egyhazasradoc,25,HU,47.094071,16.635648,0.534683,/1622/8749/1/,True,False,/1622/8749/1/,1622,1,22,19,False,/1622/8749/
1,5,Sagarmatha Zone,Sagarmatha_Zone,11,NP,27.323826,86.741637,0.866882,/3988/5/,True,False,/3988/5/,3988,16,16,16,False,/3988/


In [6]:
#Extracting hotel table and creating local csv
hotel = read_sql("select * from toy1.hotel", con=connection)
hotel.to_csv('hotel.csv', index = False)
hotel.head(2)

,hotel_id,hotel_name,rating,hotel_file_name,modified_on,created_on,number_of_reviews,cleanliness_rating,service_rating,facilities_rating,...,place_id,wk_place_id,themes,min_rate_usd,short_stay_price_min,short_stay_price_max,cheapest_check_in,cheapest_check_out,address,image_sprite_all
0,1040834,Concorde Hotel Lousberg,3,Concorde_Hotel_Lousberg,2016-02-12 07:58:23,2006-12-07 23:34:07,150,8.19,8.31,9.08,...,22898,22898,5|6|7|,96.2130,0.0,0,2016-06-20,2016-06-21,Saarstrasse 108,"155326784,460,353,ESK|155326785,375,500,ESK|15..."
1,1040983,Camino Real Acapulco Diamante,5,Camino_Real_Acapulco_Diamante,2016-01-11 16:00:08,2006-12-07 23:34:11,1205,8.59,8.19,7.92,...,55086,55086,7|8|9|13|14|,90.0561,178.5,255,2016-06-25,2016-06-26,Carretera Escénica Km.14 Baja Catita 18,"294479649,333,500,OSV|294479650,333,500,OSV|29..."


In [7]:
#Extracting lead table and creating local csv
#leadCount = read_sql("select count(1) from toy1.lead", con=connection)
#lead1 = read_sql("select * from toy1.lead limit leadCount/2", con=connection)
#lead2 = read_sql("select * from toy1.lead offset leadCount/2", con=connection)

#Prior to execution pls note: this request takes in excess of 20 minutes to process (subject to n/w latency) and unexpected source server connection failure
lead = read_sql("select * from toy1.lead", con=connection) 
lead.to_csv('lead.csv', index = False)
lead.head(2)

,lead_id,date,total_usd,check_in,check_out,room_description,nb_rooms,hotel_id
0,4031968,2016-01-01,1003.90,2015-12-31,2016-01-03,2,1,1040995
1,4049612,2016-01-01,233.61,2016-01-06,2016-01-08,2,1,1040953


In [8]:
#Credentials to establish Object Storage connection

credentials_1 = {
  'auth_url':'https://identity.open.softlayer.com',
  'project':'object_storage_4f7c12d0_b952_450d_b479_df6fdbb30d48',
  'project_id':'a52717a052f343a1b9020e4160a61913',
  'region':'dallas',
  'user_id':'c8d82fa9a27147ba8a9c1d984f5185d3',
  'domain_id':'68c916a652e2434c81421952c4c05cc9',
  'domain_name':'1434337',
  'username':'member_157b57e8fa7b7b37664471ffe4919c0ba44b9c4f',
  'password':"""L7{xz9VVCq^6k80s""",
  'container':'FindHotelProjecttemp',
  'tenantId':'undefined',
  }

In [9]:
#Function to connect Object Storage and write files

from io import BytesIO  
import requests  
import json  
import pandas as pd

def put_file(credentials, local_file_name):  
    """This functions returns a StringIO object containing
    the file content from Bluemix Object Storage V3."""
    f = open(local_file_name,'r')
    my_data = f.read()
    url1 = ''.join(['https://identity.open.softlayer.com', '/v3/auth/tokens'])
    data = {'auth': {'identity': {'methods': ['password'],
            'password': {'user': {'name': credentials['username'],'domain': {'id': credentials['domain_id']},
            'password': credentials['password']}}}}}
    headers1 = {'Content-Type': 'application/json'}
    resp1 = requests.post(url=url1, data=json.dumps(data), headers=headers1)
    resp1_body = resp1.json()
    for e1 in resp1_body['token']['catalog']:
        if(e1['type']=='object-store'):
            for e2 in e1['endpoints']:
                        if(e2['interface']=='public'and e2['region']=='dallas'):
                            url2 = ''.join([e2['url'],'/', credentials['container'], '/', local_file_name])
    s_subject_token = resp1.headers['x-subject-token']
    headers2 = {'X-Auth-Token': s_subject_token, 'accept': 'application/json'}
    resp2 = requests.put(url=url2, headers=headers2, data = my_data )
    print "File write successful"

In [ ]:
# Writing to object storage

#put_file(credentials_1,'place.csv') 
#put_file(credentials_1,'hotel.csv')
#put_file(credentials_1,'lead.csv')